In [1]:
from bayes_sensor import *
from config import VALID_CONFIG
from pprint import pprint
DEFAULT_PROBABILITY_THRESHOLD = 0.5

I wish to describe how the bayesian_sensor operates, lets do an investigation

### Code references
* https://github.com/home-assistant/home-assistant/blob/dev/homeassistant/components/binary_sensor/bayesian.py code
* https://home-assistant.io/components/binary_sensor.bayesian/ docs
* https://github.com/jlmcgehee21/smart_hass#binary-bayes-introspection HA sensor author script for working with bayes sensor  
* https://github.com/home-assistant/home-assistant/tree/a1f238816b6130aee2ac88fe9da54ba8f65225f3 Very early home-assistant commit to better understand HA architechture


### Bayes references
* https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/02-Discrete-Bayes.ipynb Recommended reading
* https://en.wikipedia.org/wiki/Bayes%27_theorem wikipedia on Bayes theorem
* https://en.wikipedia.org/wiki/Bayesian_inference wikipedia on Bayesian inference

### The Bayesian sensor

So you've heard about the [bayesian sensor](https://home-assistant.io/components/binary_sensor.bayesian/) in home-assistant, and would like to know what it's good for and how to use it. The bayesian (bayes for short) sensor should be used when you want a sensor which indicates the state of a system that you cannot directly measure. Lets consider a hypothetical scenario where want a sensor that indicates whether or not someone is cooking in the kitchen. We will call the sensor **sensor.cooking** and when someone is cooking the state is ON, otherwise the state is OFF. To be consistent with the wikipedia article on [Bayes theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) lets refer to the state ON as 'event $A$' and the state OFF as 'event $\neg A$' where the $\neg$ symbol indicates 'not' $A$. I estimate that roughly 10% of the time someone is cooking, so if your were to enter the kitchen at a random time the probability ($P$) that you would find someone cooking (event $A$) is $P(A)$ = 0.1. Clearly I can then state that 90% of the time nobody will be cooking, and $P(\neg A) = 1 - P(A)$ = 0.9. We can represent this probability distribution graphically:

Lets say that I create **sensor.cooking** at a random time with no knowledge about whether someone is cooking at the time. My initial belief about the likelihood that someone is cooking ($P(A)$) is reffered to as the *prior*, as it is my belief prior to any measurement. We have previously established that the prior, $P(A)$ = 0.1. Lets say that I have built a DIY sensor for detecting cooking smells and through experimentation I have concluded that it measures a true-positive ON 70% of the time when someone is cooking. I call the ON state of this DIY sensor 'event $B$' and can now state the likelihood that this sensor is ON *given* that someone is cooking is 70%. In algebraic notation, *given* is represented by the $|$ symbol and I can now state that the likelihood $P(A|B)$ = 0.7. However this DIY sensor is not perfect and measures a false-positive ON 10% of the time. We capture this false-positive likelihood in the expression $P(B| \neg A)$ = 0.1. 

It turns out that we now have enough information to calculate the propbability that we really care about, $P(A|B)$ which is the probability that someone is cooking *given* that we measured an ON with the DIY sensor. This probability is referred to as the *posterior* probability, as is is calculated after (or posterior to) a the measurement. The posterior is calculated using the Bayes forumla, where in words:

$$\mathtt{posterior} = \frac{\mathtt{likelihood} \times \mathtt{prior}}{\mathtt{normalisation}}$$ 

The [full algebraic expression](https://en.wikipedia.org/wiki/Bayes%27_theorem) is:

$$P(A|B) = \frac{P(B|A)\,P(A)}{ P(B|A) P(A) + P(B| \neg A) P(\neg A)}\cdot$$

Bayes formula is implemented in the Home-assistant [bayesian_sensor](https://home-assistant.io/components/binary_sensor.bayesian/) in the function update_probability():

In [2]:
def update_probability(prior, prob_true, prob_false):
    """Update probability using Bayes' rule."""
    numerator = prob_true * prior
    denominator = numerator + prob_false * (1 - prior)
    probability = numerator / denominator
    return probability

For bayesian sensors we typically define a cutoff probability of 50% ($P$ = 0.5), below which we consider the sensor to be OFF, and obove which the sensor is ON. Before any measurments we only have our prior to estimate the state of the sensor, and since our prior is below the cutoff (0.1 < 0.5) the initial state of **sensor.cooking** is OFF. 


Lets now imagine we've made a single measurement using the DIY sensor and its state is ON. We know that for a single reading the likelihood $P(B|A)$ = 0.7. To use the same variable names as update_probability(), our complete set of inputs are; the prior $P(A)$ = 0.1, the true positive likelihood prob_true = $P(B|A)$ = 0.7, and the false positive likelihood prob_false = $P(B| \neg A)$ = 0.1. Lets plug these into update_probability() (which is Bayes formula) and calculate the posterior probability ($P(A|B)$):

In [3]:
prior = 0.1
prob_true = 0.7
prob_false = 0.1
posterior = update_probability(prior, prob_true, prob_false)
print(round(posterior,2))

0.44


We find that after a single reading the posterior is below the 50% threshold and **sensor.cooking** remains OFF. 
However the posterior probability of 0.44 now becomes the prior for the next measurement. Lets say that the second measurement with the DIY sensor also reads ON, and calculate the new posterior probability:

In [4]:
posterior = update_probability(0.44, prob_true, prob_false)
print(round(posterior,2))

0.85


And great news, with only two measurements the posterior probability is above the 50% threshold and **sensor.cooking** is now ON. 

Lets now consider a second hypothetical scenarior, identical to the first scenario except that the DIY sensor is much less sensitive, and only indicates cooking 20% of the time. However it maintains the 10% false-positive rate. How many consecutive ON measurements from the DIY sensor are required to trigger **sensor.cooking** to ON?

In [5]:
prior = 0.1
prob_true = 0.2
prob_false = 0.1
posterior = prior # Our initial conditions
while posterior < 0.5:
    posterior = update_probability(posterior, prob_true, prob_false)
    print(posterior)

0.18181818181818182
0.3076923076923077
0.47058823529411764
0.64


We find that four consecutive ON measurements are required to trigger **sensor.cooking** to ON.

Now, how do we update our posterior if we receive a negative reading?

How about combining the readings from several sensors?

## Bayesian sensor
Lets now import the actual sensor used in HA. This sensor updates as new observations come in, sequentially updating the prior.

In [6]:
VALID_CONFIG

{'device_class': 'binary_device',
 'name': 'in_bed',
 'observations': [{'entity_id': 'sensor.bedroom_motion',
   'platform': 'state',
   'prob_given_true': 0.5,
   'to_state': 'on'},
  {'entity_id': 'sun.sun',
   'platform': 'state',
   'prob_given_true': 0.7,
   'to_state': 'below_horizon'}],
 'prior': 0.25,
 'probability_threshold': 0.95}

Create a bayesian sensor to explore properties

In [7]:
b_sensor = setup_platform(VALID_CONFIG)

updates are handeled through async_threshold_sensor_state_listener()

In [8]:
pprint(vars(b_sensor))

{'_deviation': False,
 '_device_class': 'binary_device',
 '_name': 'in_bed',
 '_observations': [{'entity_id': 'sensor.bedroom_motion',
                    'id': 0,
                    'platform': 'state',
                    'prob_given_true': 0.5,
                    'to_state': 'on'},
                   {'entity_id': 'sun.sun',
                    'id': 1,
                    'platform': 'state',
                    'prob_given_true': 0.7,
                    'to_state': 'below_horizon'}],
 '_probability_threshold': 0.95,
 'current_obs': OrderedDict(),
 'entity_obs': {'sensor.bedroom_motion': [{'entity_id': 'sensor.bedroom_motion',
                                           'id': 0,
                                           'platform': 'state',
                                           'prob_given_true': 0.5,
                                           'to_state': 'on'},
                                          {'entity_id': 'sun.sun',
                                           'id

In [9]:
b_sensor.device_state_attributes

{'observations': [], 'probability': 0.25, 'probability_threshold': 0.95}

In [10]:
b_sensor.state

'off'

In [11]:
b_sensor.entity_obs.keys()

dict_keys(['sensor.bedroom_motion', 'sun.sun'])

In [12]:
b_sensor.current_obs.values()

odict_values([])